In [ ]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd

# Read the data
df = pd.read_csv('/Users/huiyisang/Desktop/Modeling A2/hotels.csv')

# Create model
model = gb.Model("Hotel_Staff_Optimization_Relaxed")

# Constants
num_attendants = 8
hourly_wage = 25
regular_hours = 8
max_overtime = 2
sq_ft_limit = 3500
floor_bonus = 75

# Get unique floors and create floor mapping
floors = sorted(df['Floor'].unique())
floor_to_idx = {floor: idx for idx, floor in enumerate(floors)}
num_floors = len(floors)

# Create variables - now continuous instead of binary
# xir between 0 and 1 for room assignments
x = model.addVars(num_attendants, len(df), lb=0, ub=1, name="room_assignment")

# fik between 0 and 1 for floor assignments
f = model.addVars(num_attendants, num_floors, lb=0, ub=1, name="floor_assignment")

# oi between 0 and 1 for overtime hours
o = model.addVars(num_attendants, max_overtime, lb=0, ub=1, name="overtime")

# si between 0 and 1 for square footage violations
s = model.addVars(num_attendants, lb=0, ub=1, name="sqft_violation")

# Objective function remains the same
obj = (sum(regular_hours * hourly_wage for i in range(num_attendants)) +  # Base pay
       sum(hourly_wage * 1.5 * (o[i,0] + o[i,1]) for i in range(num_attendants)) +  # Overtime
       sum(floor_bonus * (sum(f[i,k] for k in range(num_floors)) - 2) 
           for i in range(num_attendants)) +  # Floor bonus
       sum(regular_hours * 2 * hourly_wage * s[i] for i in range(num_attendants)))  # Square footage violation

model.setObjective(obj, GRB.MINIMIZE)

# Constraints remain the same
# Each room must be assigned to exactly one attendant
for j in range(len(df)):
    model.addConstr(sum(x[i,j] for i in range(num_attendants)) == 1)

# Link floor assignments to room assignments
for i in range(num_attendants):
    for floor in floors:
        floor_idx = floor_to_idx[floor]
        room_indices = df[df['Floor'] == floor].index
        model.addConstr(sum(x[i,j] for j in room_indices) <= len(room_indices) * f[i,floor_idx])
        model.addConstr(sum(x[i,j] for j in room_indices) >= f[i,floor_idx])

# Square footage constraint
M = sum(df.Square_Feet)
for i in range(num_attendants):
    model.addConstr(sum(df.loc[j,'Square_Feet'] * x[i,j] for j in range(len(df))) <= 
                    sq_ft_limit + M * s[i])

# Overtime constraints
for i in range(num_attendants):
    model.addConstr(sum(df.loc[j,'Cleaning_Time_Hours'] * x[i,j] for j in range(len(df))) <= 
                    regular_hours + o[i,0] + o[i,1])
    model.addConstr(o[i,1] <= o[i,0])

# Floor limit constraint
for i in range(num_attendants):
    model.addConstr(sum(f[i,k] for k in range(num_floors)) >= 2)
    model.addConstr(sum(f[i,k] for k in range(num_floors)) <= 4)

# Optimize
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Cost: ${model.objVal:.2f}")
    print(f"Big M value used: {M:.2f}")
    
    # Count overtime hours
    total_overtime = sum(o[i,j].x for i in range(num_attendants) for j in range(max_overtime))
    print(f"Total Overtime Hours: {total_overtime}")
    
    # Count floor violations
    floor_violations = sum(max(0, sum(f[i,k].x for k in range(num_floors)) - 2) 
                         for i in range(num_attendants))
    print(f"Total Floor Violations: {floor_violations}\n")
    
    print("Detailed Attendant Statistics:")
    print("-" * 80)
    print(f"{'Attendant':^10} {'Hours':^10} {'Rooms':^10} {'Sq Feet':^12} {'Floors':^10} {'Overtime':^10}")
    print("-" * 80)
    
    for i in range(num_attendants):
        # Calculate statistics for each attendant
        hours = sum(df.loc[j,'Cleaning_Time_Hours'] * x[i,j].x for j in range(len(df)))
        rooms = sum(x[i,j].x for j in range(len(df)))
        sq_feet = sum(df.loc[j,'Square_Feet'] * x[i,j].x for j in range(len(df)))
        floors = sum(f[i,k].x for k in range(num_floors))
        overtime = sum(o[i,j].x for j in range(max_overtime))
        
        print(f"{i+1:^10d} {hours:^10.2f} {rooms:^10.1f} {sq_feet:^12.1f} {int(floors):^10.0f} {overtime:^10.1f}")
    
    print("-" * 80)

In [6]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd

# Read the data
df = pd.read_csv('/Users/huiyisang/Desktop/Modeling A2/hotels.csv')

# Create model
model_relaxed = gb.Model("Hotel_Staff_Optimization_Relaxed")

# Constants
num_attendants = 8
hourly_wage = 25
regular_hours = 8
max_overtime = 2
sq_ft_limit = 3500
floor_bonus = 75


# Get unique floors and create floor mapping
floors = sorted(df['Floor'].unique())
floor_to_idx = {floor: idx for idx, floor in enumerate(floors)}
num_floors = len(floors)

# Create variables (relaxed to continuous)
x = model_relaxed.addVars(num_attendants, len(df), vtype=GRB.CONTINUOUS, lb=0, ub=1, name="room_assignment")
f = model_relaxed.addVars(num_attendants, num_floors, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="floor_assignment")
o = model_relaxed.addVars(num_attendants, max_overtime, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="overtime")
s = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="sqft_violation")

# Objective function
obj_relaxed = (
    sum(regular_hours * hourly_wage for i in range(num_attendants)) +  # Base pay
    sum(hourly_wage * 1.5 * (o[i,0] + o[i,1]) for i in range(num_attendants)) +  # Overtime
    sum(floor_bonus * (sum(f[i,k] for k in range(num_floors)) - 2) for i in range(num_attendants)) +  # Floor bonus
    sum(regular_hours * 2 * hourly_wage * s[i] for i in range(num_attendants))  # Square footage violation
)

model_relaxed.setObjective(obj_relaxed, GRB.MINIMIZE)

# Constraints
for j in range(len(df)):
    model_relaxed.addConstr(sum(x[i,j] for i in range(num_attendants)) == 1)

for i in range(num_attendants):
    for floor in floors:
        floor_idx = floor_to_idx[floor]
        room_indices = df[df['Floor'] == floor].index
        model_relaxed.addConstr(sum(x[i,j] for j in room_indices) <= len(room_indices) * f[i,floor_idx])
        model_relaxed.addConstr(sum(x[i,j] for j in room_indices) >= f[i,floor_idx])

M = sum(df.Square_Feet)
for i in range(num_attendants):
    model_relaxed.addConstr(sum(df.loc[j,'Square_Feet'] * x[i,j] for j in range(len(df))) <= 
                            sq_ft_limit + M * s[i])

for i in range(num_attendants):
    model_relaxed.addConstr(sum(df.loc[j,'Cleaning_Time_Hours'] * x[i,j] for j in range(len(df))) <= 
                            regular_hours + o[i,0] + o[i,1])
    model_relaxed.addConstr(o[i,1] <= o[i,0])

for i in range(num_attendants):
    model_relaxed.addConstr(sum(f[i,k] for k in range(num_floors)) >= 2)
    model_relaxed.addConstr(sum(f[i,k] for k in range(num_floors)) <= 4)

# Optimize
model_relaxed.optimize()

# Print results
if model_relaxed.status == GRB.OPTIMAL:
    print(f"\nOptimal Cost (Relaxed Model): ${model_relaxed.objVal:.2f}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 316 rows, 552 columns and 2568 nonzeros
Model fingerprint: 0xe8d585fa
Coefficient statistics:
  Matrix range     [6e-01, 2e+04]
  Objective range  [4e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 308 rows, 560 columns, 2464 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6000000e+03   2.017045e-01   0.000000e+00      0s
       1    1.6605114e+03   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       0 DPushes remaining with DInf 0.0000000e+00                 0s

     477 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0